# Figyelemmechanizmusok és transzformerek

A rekurens hálózatok egyik fő hátránya, hogy egy szekvencia minden szava azonos mértékben hat a végeredményre. Ez az alapértelmezett LSTM kódoló-dekódoló modellek esetében nem optimális teljesítményt eredményez szekvencia-szekvencia feladatoknál, például a néventitás-felismerésnél vagy a gépi fordításnál. Valójában az input szekvencia bizonyos szavai gyakran nagyobb hatással vannak a kimeneti szekvenciára, mint mások.

Vegyünk például egy szekvencia-szekvencia modellt, mint a gépi fordítás. Ez két rekurens hálózattal valósul meg, ahol az egyik hálózat (**kódoló**) az input szekvenciát egy rejtett állapotba sűríti, míg a másik (**dekódoló**) ezt a rejtett állapotot bontja ki a fordított eredményre. Ennek a megközelítésnek az a problémája, hogy a hálózat végső állapota nehezen tudja megjegyezni a mondat elejét, ami gyenge modellminőséget eredményez hosszú mondatok esetén.

A **figyelemmechanizmusok** lehetőséget adnak arra, hogy súlyozzuk az egyes input vektorok kontextuális hatását az RNN kimeneti előrejelzéseire. Ez úgy valósul meg, hogy rövidítéseket hozunk létre az input RNN köztes állapotai és a kimeneti RNN között. Ily módon, amikor a $y_t$ kimeneti szimbólumot generáljuk, figyelembe vesszük az összes input rejtett állapotot $h_i$, különböző súlykoefficiensekkel $\alpha_{t,i}$. 

![Kép egy kódoló/dekódoló modellről additív figyelemréteggel](../../../../../translated_images/hu/encoder-decoder-attention.7a726296894fb567.webp)
*A kódoló-dekódoló modell additív figyelemmechanizmussal [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) alapján, idézve [ebből a blogbejegyzésből](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

A figyelem mátrix $\{\alpha_{i,j}\}$ azt mutatja, hogy az egyes input szavak milyen mértékben járulnak hozzá egy adott szó generálásához a kimeneti szekvenciában. Az alábbiakban egy ilyen mátrix példáját láthatjuk:

![Kép egy mintaillesztésről, amelyet az RNNsearch-50 talált, Bahdanau - arviz.org](../../../../../translated_images/hu/bahdanau-fig3.09ba2d37f202a6af.webp)

*Ábra [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (3. ábra) alapján*

A figyelemmechanizmusok felelősek a természetes nyelvfeldolgozás jelenlegi vagy közel jelenlegi csúcsteljesítményéért. A figyelem hozzáadása azonban jelentősen növeli a modell paramétereinek számát, ami méretezési problémákat okozott az RNN-eknél. Az RNN-ek méretezésének egyik kulcsfontosságú korlátja, hogy a modellek rekurzív jellege megnehezíti az edzés batch-elését és párhuzamosítását. Egy RNN-ben a szekvencia minden elemét sorrendben kell feldolgozni, ami azt jelenti, hogy nem lehet könnyen párhuzamosítani.

A figyelemmechanizmusok alkalmazása és ez a korlát vezettek a ma ismert és használt csúcsteljesítményű transzformer modellek létrejöttéhez, mint például a BERT vagy az OpenGPT3.

## Transzformer modellek

Ahelyett, hogy az előző előrejelzések kontextusát továbbítanák a következő értékelési lépésbe, a **transzformer modellek** **pozíciós kódolásokat** és **figyelmet** használnak, hogy megragadják az adott input kontextusát egy megadott szövegablakon belül. Az alábbi kép bemutatja, hogyan képesek a pozíciós kódolások és a figyelem megragadni a kontextust egy adott ablakon belül.

![Animált GIF, amely bemutatja, hogyan történik az értékelés a transzformer modellekben.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Mivel minden input pozíciót függetlenül térképeznek a kimeneti pozíciókhoz, a transzformerek jobban párhuzamosíthatók, mint az RNN-ek, ami lehetővé teszi sokkal nagyobb és kifejezőbb nyelvi modellek létrehozását. Minden figyelemfej különböző kapcsolatok megtanulására használható a szavak között, ami javítja a természetes nyelvfeldolgozási feladatok eredményét.

## Egyszerű transzformer modell építése

A Keras nem tartalmaz beépített transzformer réteget, de saját magunk is felépíthetjük. Mint korábban, most is az AG News adathalmaz szövegklasszifikációjára fogunk összpontosítani, de érdemes megjegyezni, hogy a transzformer modellek a legjobb eredményeket a nehezebb NLP feladatoknál mutatják.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Az új rétegeknek a Kerasban alosztályozniuk kell a `Layer` osztályt, és implementálniuk kell a `call` metódust. Kezdjük a **Positional Embedding** réteggel. [Az alábbi kódot az hivatalos Keras dokumentációból](https://keras.io/examples/nlp/text_classification_with_transformer/) fogjuk használni. Feltételezzük, hogy az összes bemeneti szekvenciát `maxlen` hosszúságúra töltjük fel.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Ez a réteg két `Embedding` rétegből áll: az egyik a tokenek beágyazására szolgál (ahogy azt korábban megbeszéltük), a másik pedig a tokenek pozícióinak beágyazására. A tokenek pozícióit a 0-tól `maxlen`-ig terjedő természetes számok sorozataként hozzuk létre a `tf.range` segítségével, majd ezt átadjuk a beágyazási rétegnek. A két eredményül kapott beágyazási vektort összeadjuk, így létrejön a bemenet pozícióval beágyazott reprezentációja, amelynek alakja `maxlen`$\times$`embed_dim`.

Most implementáljuk a transformer blokkot. Ez a korábban definiált beágyazási réteg kimenetét fogja használni:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Most már készen állunk arra, hogy meghatározzuk a teljes transformer modellt:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Modellek

A **BERT** (Bidirectional Encoder Representations from Transformers) egy nagyon nagy, többrétegű transformer hálózat, amely *BERT-base* esetén 12 rétegből, míg *BERT-large* esetén 24 rétegből áll. A modellt először egy nagy szövegkorpuszra (WikiPedia + könyvek) tanítják be felügyelet nélküli tanulással (maszkolt szavak előrejelzése egy mondatban). Az előképzés során a modell jelentős nyelvi megértést sajátít el, amelyet később más adathalmazokkal finomhangolással lehet hasznosítani. Ezt a folyamatot **transzfer tanulásnak** nevezzük.

![kép forrása: http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/hu/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Számos Transformer architektúra létezik, például BERT, DistilBERT, BigBird, OpenGPT3 és még sok más, amelyek finomhangolhatók.

Nézzük meg, hogyan használhatunk egy előre betanított BERT modellt hagyományos szekvenciaosztályozási problémák megoldására. Az ötletet és némi kódot az [hivatalos dokumentációból](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) vesszük kölcsön.

Az előre betanított modellek betöltéséhez a **Tensorflow hubot** fogjuk használni. Először töltsük be a BERT-specifikus vektorizálót:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Fontos, hogy ugyanazt a vektorizálót használd, amelyet az eredeti hálózat betanításához használtak. A BERT vektorizáló három komponenst ad vissza:
* `input_word_ids`, amely az input mondat token számainak sorozata
* `input_mask`, amely megmutatja, hogy a sorozat mely része tartalmaz tényleges bemenetet, és melyik a kitöltés. Ez hasonló a `Masking` réteg által előállított maszkhoz
* `input_type_ids`, amelyet nyelvi modellezési feladatokhoz használnak, és lehetővé teszi két bemeneti mondat megadását egy sorozatban.

Ezután példányosíthatjuk a BERT jellemzők kinyerőjét:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Tehát a BERT réteg számos hasznos eredményt ad vissza:
* `pooled_output` az összes token átlagolásának eredménye a szekvenciában. Ezt tekinthetjük az egész hálózat intelligens szemantikai beágyazásának. Ez egyenértékű a korábbi modellünkben használt `GlobalAveragePooling1D` réteg kimenetével.
* `sequence_output` az utolsó transformer réteg kimenete (megfelel a fenti modellünkben található `TransformerBlock` kimenetének).
* `encoder_outputs` az összes transformer réteg kimenete. Mivel egy 4-rétegű BERT modellt töltöttünk be (ahogy valószínűleg a névből is sejthető, amely tartalmazza a `4_H` jelölést), ez 4 tenzort tartalmaz. Az utolsó ugyanaz, mint a `sequence_output`.

Most definiáljuk az elejétől a végéig tartó osztályozó modellt. *Funkcionális modelldefiníciót* fogunk használni, ahol meghatározzuk a modell bemenetét, majd egy sor kifejezést adunk meg a kimenet kiszámításához. A BERT modell súlyait nem tesszük taníthatóvá, és csak a végső osztályozót fogjuk tanítani:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Annak ellenére, hogy kevés a tanítható paraméter, a folyamat meglehetősen lassú, mivel a BERT jellemzők kinyerése számításigényes. Úgy tűnik, nem sikerült elfogadható pontosságot elérni, akár a nem megfelelő tanítás, akár a modellparaméterek hiánya miatt.

Próbáljuk meg feloldani a BERT súlyok zárolását, és azt is betanítani. Ehhez nagyon kicsi tanulási rátára van szükség, valamint egy körültekintőbb tanítási stratégiára **warmup**-pal, az **AdamW** optimalizáló használatával. Az optimalizáló létrehozásához a `tf-models-official` csomagot fogjuk használni:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Amint láthatod, a tanítás meglehetősen lassan halad – de érdemes lehet kísérletezni, és néhány epochon (5-10) keresztül tanítani a modellt, hogy összehasonlítsd az eredményeket az általunk korábban alkalmazott megközelítésekkel.

## Huggingface Transformers könyvtár

Egy másik nagyon elterjedt (és valamivel egyszerűbb) módja a Transformer modellek használatának a [HuggingFace csomag](https://github.com/huggingface/), amely egyszerű építőelemeket biztosít különböző NLP feladatokhoz. Ez elérhető mind Tensorflow, mind PyTorch számára, amelyek szintén nagyon népszerű neurális hálózati keretrendszerek. 

> **Note**: Ha nem érdekel, hogyan működik a Transformers könyvtár, akkor átugorhatod a notebook végét, mert nem fogsz lényegesen eltérő dolgokat látni ahhoz képest, amit korábban csináltunk. Ugyanazokat a lépéseket fogjuk ismételni a BERT modell tanításához, csak egy másik könyvtárat és lényegesen nagyobb modellt használva. Ezért a folyamat meglehetősen hosszú tanítást igényel, így lehet, hogy csak a kódot szeretnéd átnézni.

Nézzük meg, hogyan oldható meg a problémánk a [Huggingface Transformers](http://huggingface.co) segítségével.


Először ki kell választanunk a modellt, amelyet használni fogunk. A beépített modellek mellett a Huggingface rendelkezik egy [online modell-adattárral](https://huggingface.co/models), ahol a közösség által előre betanított modellek széles választékát találhatod. Ezeket a modelleket mindössze a modell nevének megadásával lehet betölteni és használni. Az összes szükséges bináris fájl automatikusan letöltésre kerül.

Bizonyos esetekben szükséged lehet arra, hogy saját modelleket tölts be. Ilyenkor megadhatod azt a könyvtárat, amely tartalmazza az összes releváns fájlt, beleértve a tokenizáló paramétereit, a `config.json` fájlt a modell paramétereivel, bináris súlyokat stb.

A modell nevéből létrehozhatjuk mind a modellt, mind a tokenizálót. Kezdjük a tokenizálóval:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

A `tokenizer` objektum tartalmazza az `encode` függvényt, amely közvetlenül használható szöveg kódolására:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Használhatjuk a tokenizálót arra, hogy egy sorozatot olyan módon kódoljunk, amely alkalmas a modellnek való átadásra, azaz tartalmazza a `token_ids`, `input_mask` mezőket stb. Megadhatjuk azt is, hogy Tensorflow tenzorokat szeretnénk, ha megadjuk a `return_tensors='tf'` argumentumot:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Ebben az esetben egy előre betanított BERT modellt fogunk használni, amelynek neve `bert-base-uncased`. Az *uncased* azt jelzi, hogy a modell nem érzékeny a kis- és nagybetűkre.

A modell betanításakor tokenizált szekvenciát kell bemenetként megadnunk, ezért egy adatfeldolgozási folyamatot fogunk kialakítani. Mivel a `tokenizer.encode` egy Python függvény, ugyanazt a megközelítést fogjuk alkalmazni, mint az előző egységben, azaz a `py_function` hívásával:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Most már betölthetjük a tényleges modellt a `BertForSequenceClassification` csomag használatával. Ez biztosítja, hogy a modellünk már rendelkezik a szükséges architektúrával az osztályozáshoz, beleértve a végső osztályozót is. Figyelmeztető üzenetet fogsz látni, amely azt jelzi, hogy a végső osztályozó súlyai nincsenek inicializálva, és a modell előzetes tanítást igényel - ez teljesen rendben van, mivel pontosan ezt fogjuk tenni!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Amint látható a `summary()` alapján, a modell közel 110 millió paramétert tartalmaz! Feltételezhetően, ha egyszerű osztályozási feladatot szeretnénk egy viszonylag kis adathalmazon, nem akarjuk betanítani a BERT alapréteget:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Most már készen állunk az edzés megkezdésére!

> **Megjegyzés**: A teljes méretű BERT modell betanítása rendkívül időigényes lehet! Ezért csak az első 32 batch-re fogjuk betanítani. Ez csupán arra szolgál, hogy bemutassuk, hogyan van beállítva a modell betanítása. Ha szeretnéd kipróbálni a teljes méretű betanítást, egyszerűen távolítsd el a `steps_per_epoch` és `validation_steps` paramétereket, és készülj fel a várakozásra!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Ha növeled az iterációk számát, elég sokáig vársz, és több epochon keresztül tanítasz, akkor várhatóan a BERT osztályozás adja a legjobb pontosságot! Ez azért van, mert a BERT már eleve elég jól érti a nyelv szerkezetét, és nekünk csak a végső osztályozót kell finomhangolnunk. Azonban, mivel a BERT egy nagy modell, az egész tanítási folyamat sok időt vesz igénybe, és komoly számítási kapacitást igényel! (GPU, és lehetőleg több is).

> **Megjegyzés:** Példánkban az egyik legkisebb előre betanított BERT modellt használtuk. Vannak nagyobb modellek is, amelyek valószínűleg jobb eredményeket hoznak.


## Legfontosabb tanulság

Ebben az egységben megismerkedtünk a legújabb **transformer** alapú modellarchitektúrákkal. Alkalmaztuk őket szövegklasszifikációs feladatunkra, de hasonlóan a BERT modellek használhatók entitáskinyerésre, kérdés-válaszolásra és más NLP feladatokra is.

A transformer modellek jelenleg az NLP csúcstechnológiáját képviselik, és a legtöbb esetben ezekkel érdemes először kísérletezni, amikor egyedi NLP megoldásokat valósítasz meg. Ugyanakkor rendkívül fontos megérteni az ebben a modulban tárgyalt visszatérő neurális hálózatok alapelveit, ha fejlettebb neurális modelleket szeretnél építeni.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
